In [ ]:
#default_exp cam

# CAM

> Class activation maps for time series motions. The methods coded here are an adaptation of [the CAM methods from the ai-fast-track/timeseries library.](https://github.com/ai-fast-track/timeseries/blob/master/timeseries/cam.py)

In [ ]:
#export
from timeseries.all import *
from fastai.imports import *
from fastai.basics import *
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.cm as cm

In [ ]:
#export
@delegates(LineCollection.__init__)
def show_cam_motion(batch, model, layer=5, func_cam=cam_acts, reduction='mean',force_scale=True, scale_range=(0, 1),                        cmap="Spectral_r", linewidth=4, linestyles='solid', alpha=1.0, scatter=False, i2o=noop,
                        figsize=(13, 4), confidence=None, savefig=None, ylim=None, **kwargs):
    'Compute CAM using `func_cam` function, and plot a batch of colored time series `tseries`. The colors correspond to the        scaled CAM values. Each time series is plotted on a separate figure'

    # batch use-cases
    # batch can be either:
    #   - a real batch meaning a tuple of (a list of tseries, a list of y)
    #   - a list of tuples (tseries, y) that we build from a dataset (a dataset item is a tuple of (tseries, y)
    #   - a single dataset item meaning a a tuple of (tseries, y)

    # print(f'Confidence: {confidence}')
    if figsize==None: figsize=(13,4)
    if not isinstance(batch, list):
        if len(batch[0].shape)==3:   # a real batch meaning a tuple of (a list of tseries, a list of y)
            b = itemize(batch)
        elif len(batch[0].shape)==2: # a single dataset item meaning a a tuple of (tseries, y)m
            b = [batch]
    else:
        b = batch # a list of tuples (tseries, y) that we build from a dataset (a dataset item is a tuple of (tseries, y)ch
    n_samples = len(b)

    # b = _listify(b)
    tseries_list, acts_list, y_list, ts_min_max, acts_min_max = CAM_batch_compute(b, model, layer=layer, func_cam=func_cam,                                              reduction=reduction, force_scale=force_scale, scale_range=scale_range, **kwargs)
    levels = 254
    cmap = plt.get_cmap(cmap, lut=levels) #seismic
    if force_scale==True:
        colors = cmap(np.linspace(scale_range[0], scale_range[1], levels))
        norm = BoundaryNorm(np.linspace(scale_range[0], scale_range[1], levels+1), len(colors))
    else:
        colors = cmap(np.linspace(acts_min_max[0], acts_min_max[1], levels))
        norm = BoundaryNorm(np.linspace(acts_min_max[0], acts_min_max[1], levels+1), len(colors))


    # Grid
    nrows = int(math.sqrt(n_samples))
    ncols = int(np.ceil(n_samples/nrows))
    fig,axs = subplots(nrows, ncols, figsize=figsize, **kwargs)
    axs = [ax if i<n_samples else ax.set_axis_off() for i, ax in enumerate(axs.flatten())][:n_samples]
    fig.tight_layout(pad=3.0)

    # fig, axs = get_grid(4, return_fig=True, figsize=(10, 8))
    # Plot activation `acts` (superimposed on the original time series `tseries)
    lc_list=[]
    title_list=[]
    idx = 1
    for tseries, acts, y, ax in zip(tseries_list, acts_list, y_list, axs):
        #plt.figure(figsize=figsize)
        #plt.subplot(1, n_samples, idx)
        ax.scatter(tseries[0], tseries[1], cmap=cmap, c = acts)
        title = i2o(y)
        if not hasattr(func_cam, 'name'): func_cam.name = str(func_cam)
        title =  f'[{title}] - {func_cam.name} - {reduction}'
        if confidence!=None: title = f'{title}\n {confidence[idx-1]}'
        ax.set(xlim=[0, 360])
        if ylim is not None: ax.set(ylim=ylim)
        #plt.ylim(ts_min_max[0]*1.2, ts_min_max[1]*1.2)
        ax.set_title(title)
        scalarmappaple = cm.ScalarMappable(norm=norm, cmap=cmap)
        scalarmappaple.set_array(acts)
        fig.colorbar(scalarmappaple, ax=ax)
        #plt.show()
        idx += 1

    if savefig!=None: plt.savefig(savefig)

    return fig